# Dye recognition

Here is the Jupyter Notebook used for the dye recognition and the visualization tools developped.

After generating a database using creation_spectra_base.py, you can import an hyperspectral image here and use the pearson coefficient method to get the visualization we developped.


## Import

Run it two time to get the interactive plot.

In [ ]:
import csv
import numpy as np
from os import path
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import mpl_scatter_density
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as colors
import spectral.io.envi as envi
from spectral import imshow
import cv2
from scipy.stats.stats import pearsonr   


%matplotlib notebook

# Affichage du PCA sur un graphe 2D
# "Viridis-like" colormap with white background
white_viridis = LinearSegmentedColormap.from_list('white_viridis', [
    (0, '#ffffff'),
    (1e-20, '#440053'),
    (0.2, '#404388'),
    (0.4, '#2a788e'),
    (0.6, '#21a784'),
    (0.8, '#78d151'),
    (1, '#fde624'),
], N=256)

## Input for the dyes studies


In [ ]:
# File path
filename = "D://Spectral//Aubusson/Fleur//"

# Name of the file, usually "001"
img_nbr = "001"

# Name of the database for reference spectra used
database_name = "red_dyes"
#-----------------------------------------------
# Region of interest definition
point_1 = [616, 912]
point_2 = [1274,1391]
#-----------------------------------------------

## Import the database of reference spectra

In [ ]:
reader = csv.reader(open('data//databases//'+database_name+'.csv', mode='r'))

pigments_dic = {}
for row in reader:
    pigments_dic[row[0]]=[float(i) for i in row[1:]]

pigments = np.array([pigments_dic[key] for key in pigments_dic.keys()])
label = np.array([key for key in pigments_dic.keys()])

bands = np.array(np.genfromtxt('bands_long.csv', delimiter=','))


In [ ]:
plt.figure()
i = 0
for pig in pigments :
    plt.plot(bands, pig, label = label[i])
    i = i +1
    
plt.legend(loc='upper right')
plt.title('Reference spectra')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Reflectance')
plt.show()


## Import the hyperspectral image studied

In [ ]:
if path.exists(filename+'array_white.txt') == False :
    # Creation of white and dark reference
    white_nparr = np.array(envi.open(filename+"WHITEREF_"+img_nbr+".hdr", filename+"WHITEREF_"+img_nbr+".raw").load())

    white_val = []
    for i in range(840):
        white_val.append(np.mean(white_nparr[:,:, i]))
    file1 = open(filename+'array_white.txt',"w")
    np.savetxt(file1, [white_val])
    file1.close()

    dark_nparr = np.array(envi.open(filename+"DARKREF_"+img_nbr+".hdr", filename+"DARKREF_"+img_nbr+".raw").load())
    print(np.min(dark_nparr))
    dark_val = []
    for i in range(840):
        dark_val.append(np.mean(dark_nparr[:,:, i]))
    dark_val[0] = dark_val[1]
    file1 = open(filename+'array_dark.txt',"w")
    np.savetxt(file1, [dark_val])
    file1.close()

    del white_nparr, white_val, dark_nparr, dark_val

# Import the image    
data_nparr = (envi.open(filename+img_nbr+".hdr", filename+img_nbr+".raw").load())[100:-100, 100:-100,:]
        
# Import reference
white_file = open(filename+'array_white.txt',"r")
white_val = np.loadtxt(white_file)

dark_file = open(filename+'array_dark.txt',"r")
dark_val = np.loadtxt(dark_file)
dark_val[0] = dark_val[1]

cut_corrected_nparr = data_nparr[point_1[0]:point_2[0] , point_1[1]:point_2[1],:]

cut_corrected_nparr = np.divide(
        np.subtract(cut_corrected_nparr, dark_val),
        np.subtract(white_val, dark_val))

# Plot the image
imshow(cut_corrected_nparr, (73*4, 52*4, 23*4))
plt.axis('off')

In [ ]:
# Bilateral filtering
c_map = cut_corrected_nparr.astype(np.float32, copy=False)
for i in range(len(cut_corrected_nparr[0][0])):
    img = np.array(cv2.bilateralFilter(c_map[:,:,i], 20, 0.1, 45))
    cut_corrected_nparr[:,:,i] = img
plt.tight_layout()

imshow(cut_corrected_nparr, (73*4, 52*4, 23*4))
plt.axis('off')
plt.tight_layout()

## PCA on the pearson coefficients and visualization

In [ ]:
# Creating the correlation maps
correlation_maps = np.zeros((np.shape(cut_corrected_nparr)[0],
                             np.shape(cut_corrected_nparr)[1],
                             np.shape(pigments)[0]))

for x in range(np.shape(correlation_maps)[0]):
            for y in range(np.shape(correlation_maps)[1]):
                for z in range(np.shape(correlation_maps)[2]):
                    b = cut_corrected_nparr[x, y,:]/sum(cut_corrected_nparr[x, y,:])
                    a = np.array(pigments[z, :])
                    corr_x, corr_y = pearsonr(a,b)
                    correlation_maps[x, y, z] = corr_x

In [ ]:
# Doing the PCA on our data
data = np.reshape(correlation_maps,
                   (np.shape(correlation_maps)[0]*np.shape(correlation_maps)[1],np.shape(correlation_maps)[2])).T

pca = PCA(n_components=2)
pca.fit(data)

img_pca = pca.components_[:,:np.shape(correlation_maps)[0]*np.shape(correlation_maps)[1]]

In [ ]:
vmax = 70

# Interactive plot: 
class Cursor:
    def __init__(self, fig, ax, ax2, ax3, img, data, spect_img, rad):
        self.fig = fig
        self.ax = ax
        self.ax2 = ax2
        self.ax3 = ax3
        self.ly = ax.axvline(color='k')  # the vert line
        self.lx = ax.axhline(color='k')  # the vert line
        self.data = data
        self.img = img
        self.spect_img = spect_img
        self.value = 5
        self.rad = rad

        # text location in axes coords
        self.txt = ax.text(0.05, 0.95, '', transform=ax.transAxes)

    def mouse_move(self, event):
        if not event.inaxes:
            return

        x = event.xdata
        y = event.ydata
        # update the line positions
        self.ly.set_xdata(x)
        self.lx.set_ydata(y)

        # Image avec rouge
        self.txt.set_text('value= x:'+str(round(x,5))+' y:'+str(round(y,5)))
        img2 = np.zeros((np.shape(self.img)[0], np.shape(self.img)[1], 3))
        img2[np.sqrt((self.data[:,:,0]-x)**2+(self.data[:,:,1]-y)**2) < self.rad] = [1, 0, 0]
        img2[np.sqrt((self.data[:,:,0]-x)**2+(self.data[:,:,1]-y)**2) >= self.rad] = self.img[np.sqrt((self.data[:,:,0]-x)**2+(self.data[:,:,1]-y)**2) >= self.rad]
        
        self.ax2 = fig.add_subplot(2,2,3)
        self.ax2 = plt.imshow(img2)
        self.ax2 = plt.axis('off')
        
        # Spectre normalisé 
        pca_data = np.reshape(self.data, (np.shape(self.data)[0]*np.shape(self.data)[1], 2))
        mask = np.ma.masked_where(np.sqrt((pca_data[:,0]-x)**2+(pca_data[:,1]-y)**2) > self.rad, pca_data[:,0])
        mask = np.reshape(0*mask+1, (np.shape(self.img)[0], np.shape(self.img)[1]))
        spectrum = []
        for i in range(np.shape(self.spect_img)[2]):
            spectrum.append(np.mean(mask*self.spect_img[:,:,i]))
        self.ax3 = fig.add_subplot(2,2,4)
        self.ax3 = plt.cla()
        self.ax3 = plt.plot(bands, spectrum)
        self.ax3 = plt.grid()
        self.ax3 = plt.title('Selected spectrum')
        
        self.ax.figure.canvas.draw()
        
rgb_img = np.zeros((np.shape(cut_corrected_nparr)[0], np.shape(cut_corrected_nparr)[1], 3))
rgb_img[:,:,0] = cut_corrected_nparr[:,:,74*4]
rgb_img[:,:,1] = cut_corrected_nparr[:,:,52*4]
rgb_img[:,:,2] = cut_corrected_nparr[:,:,23*4]

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(2, 1, 1, projection='scatter_density')
ax.grid()
density = ax.scatter_density(img_pca[:][0], img_pca[:][1], cmap=white_viridis, vmin= 0, vmax=vmax, alpha = 1)

ax2 = fig.add_subplot(2,2,4)

ax3 = fig.add_subplot(2,2,3)


pca2img = np.zeros((np.shape(rgb_img)[0],np.shape(rgb_img)[1], 2))
pca2img[:,:,0] = np.reshape(img_pca[:][0],
                   (np.shape(rgb_img)[0],np.shape(rgb_img)[1]))
pca2img[:,:,1] = np.reshape(img_pca[:][1],
                   (np.shape(rgb_img)[0],np.shape(rgb_img)[1]))



cursor = Cursor(fig, ax, ax2, ax3, rgb_img, pca2img, cut_corrected_nparr, 0.0002)
fig.canvas.mpl_connect('button_press_event', cursor.mouse_move)



plt.tight_layout()
plt.show()